In [33]:
import matplotlib.pyplot as plt
import rloutrading as rlou
from importlib import reload

In [31]:
reload(rlou)

<module 'rloutrading' from '/Users/gerardoduran/repos/projects/rl-ou-process/rloutrading.py'>

In [49]:
kappa, sigma, xbar, phi, gamma, c, T, dt, A, B, C, D = 1, 0.2000, 100, 0.0100, 0.2000, 0.3000, 2, 0.1, 20, 20, 20, 20

qtr = rlou.QTrading(kappa, sigma, xbar, phi, gamma, c, T, dt, A, B, C, D)

# N-simulation episodios, time, inventory, action
qtr.simulate_reward_matrix().shape

(19, 40, 10)

In [52]:
format(10 * 500 * 40 * 20, ",")

'4,000,000'